In [ ]:
import numpy as np
import os
import pandas as pd
from watchfiles import watch
from utils.realsense import start_realsense
import shutil
from glob import glob
from analysis.analysis_h2o2 import wellsegment_h2o2, plot_titration
from time import time
import json
import matplotlib.pyplot as plt

def get_changes(titration_count):
    path_smb = r'C:\Users\scrc112\Desktop\work\biplab\OpenTron\DriveSMB'
    if os.path.exists(os.path.join(path_smb,'msi2pi_h2o2.csv')) :
        fileExist = True
        data = pd.read_csv(os.path.join(path_smb,'msi2pi_h2o2.csv'))
        if data.loc[0,'titration_count'] > titration_count :
            titration_count = data.loc[0,'titration_count']
        else :
            titration_count = False
    else :
        fileExist = False
        titration_count = False

    out = {'file': fileExist, 'titration_count' : titration_count}
    return out

def get_changes_v2():
    path_smb = r'C:\Users\scrc112\Desktop\work\biplab\OpenTron\DriveSMB'
    titration_count = False
    if os.path.exists(os.path.join(path_smb,'msi2pi_h2o2.csv')) :
        fileExist = True
        data = pd.read_csv(os.path.join(path_smb,'msi2pi_h2o2.csv'))
        if data.loc[0,'mode'] == 'pre-estimation' : save_filename = 'preestimation'

        if data.loc[0,'mode'] == 'titration':
            titration_count = data.loc[0,'titration_count']
            save_filename = 'samp_' + str(data.loc[0,'sample_id'])+ '_titration_' + str(data.loc[0,'titration_count'])+ '_'
    else :
        fileExist = False
       

    out = {'file': fileExist, 'titration_count' : titration_count, 'save_filename' : save_filename}
    return out


def get_changes_v3():
    path_smb = r'C:\Users\scrc112\Desktop\work\biplab\OpenTron\DriveSMB'
    titration_count = False
    reservoir_list_exist = False
    fileExist = False
    if os.path.exists(os.path.join(path_smb,'msi2pi_h2o2.csv')) :
        fileExist = True
        data = pd.read_csv(os.path.join(path_smb,'msi2pi_h2o2.csv'))
        titration_count = None
        save_location = None
        save_filename = None
        wells = None
        copyfile = False
        if data.loc[0,'mode'] == 'init' : 
            save_filename = 'blancplate_'

        if data.loc[0,'mode'] == 'pre-estimation' : 
            save_filename = 'preestimation_'

        if data.loc[0,'mode'] == 'titration':
            titration_count = data.loc[0,'titration_count']
            save_filename = 'samp_' + str(data.loc[0,'id'].astype('int'))+ '_titration_' + str(data.loc[0,'titration_count'].astype('int'))+ '_'
            save_location = 'samp_' + str(data.loc[0,'id'].astype('int'))
            wells = data.loc[:,'cols'].to_list()
            copyfile = True
        if os.path.exists(os.path.join(path_smb, 'reservoir_list.csv')):
            reservoir_list_exist = True
         #if data.loc[0,'mode'] == 'exit':
    #else :
        #fileExist = False
       

    out = {'file': fileExist, 
           'mode' : data.loc[0,'mode'],
           'titration_count' : titration_count, 
           'save_filename' : save_filename,
           'save_location' : save_location,
           'wells' : wells,
           'copyfile' : copyfile,
           'reservoir_list_exist' : reservoir_list_exist}
    return out

def cleanup(path, pattern):
    #path = r'C:\Users\scrc112\Desktop\work\yuan\protcol_h2o2_test\titration\samp_3'
    lst = glob(os.path.join(path,pattern+'*.png'))
    [os.remove(file) for idx,file in enumerate(lst[:-1])]


def delete_files(path_smb):
    if os.path.exists(os.path.join(path_smb,'msi2pi_h2o2.csv')) : os.remove(os.path.join(path_smb,'msi2pi_h2o2.csv'))
    if os.path.exists(os.path.join(path_smb,'reservoir_list.csv')) : os.remove(os.path.join(path_smb,'reservoir_list.csv'))

In [2]:
#################################################################################################################################
path_smb = r'C:\Users\scrc112\Desktop\work\biplab\OpenTron\DriveSMB'
path_cam = r'C:\Users\scrc112\Desktop\work\yuan\protcol_h2o2_test'

try :
    # create sub folders
    titration_base_path = os.path.join(path_cam,'titration')
    mask_path = os.path.join(path_cam,'mask')
    prest_path = os.path.join(path_cam,'preestimation')
    if not(os.path.exists(titration_base_path)) : os.mkdir(titration_base_path)
    if not(os.path.exists(mask_path)) : os.mkdir(mask_path)
    if not(os.path.exists(prest_path)) : os.mkdir(prest_path)
    
    for changes in watch(path_smb):
        print(changes)
        out = get_changes_v3()

        ############################################################################################################################################
        # blancplate imaging
        if out['file'] and out['mode'] == 'init' :
            t1 = time()
            start_realsense(fname=out['save_filename'],
                            folder=mask_path,
                            frame_interval=1,#seconds
                            stop=0.001,#hours
                            take_image=True,
                            sensitivity = 150)
            cleanup(mask_path,out['save_filename'])
            
            # save mask to mask folder
            
            # get mask from first image prior to liquid transfer
            cls = wellsegment_h2o2(mask_path, out['save_filename'] + '3.png' ,None, squeeze_fac= 0.4, col_list = [8]*12, path = mask_path)
            cls.extract() # save pickle file of sorted mask
            wellsegment_h2o2.visualize(cls.image, cls.mask)
            t2=time()
            del cls
            
            print('elapsed time on blancplate imaging : {}'.format(t2-t1))

        ###############################################################################################################################################
        # pre-estimation
        if out['file'] and out['mode'] == 'pre-estimation' and not(out['reservoir_list_exist']):
            
            t1 = time()
            start_realsense(fname=out['save_filename'],
                            folder=prest_path,
                            frame_interval=1,#seconds
                            stop=0.001,#hours
                            take_image=True,
                            sensitivity = 150)
            cleanup(prest_path,out['save_filename'])
            
            # save analysis to smb
            with open(os.path.join(path_smb,'ot2_transfer_config_h2o2.json'),'r') as f:
                data = json.load(f)
    
            tcdict = data['transfer_3']['locs']

            #cls2 = wellsegment_h2o2.load_pkl(prest_path,out['save_filename']+'3.png',os.path.join(mask_path, 'mask.pkl'))

            cls2 = wellsegment_h2o2.load_pkl(prest_path,'COSTAR-pre23.png',os.path.join(mask_path, 'mask.pkl'))
            cls2.analyze_prestimation(tcdict)
            cls2.save_to_smb_()

            t2=time()            
            print('elapsed time on pre-estimation : {}'.format(t2-t1))

        ###############################################################################################################################################
        # titration
        if out['file'] and out['mode'] == 'titration' :
            titration_path = os.path.join(titration_base_path, out['save_location'])
            if not(os.path.exists(titration_path)) : os.mkdir(titration_path)
            shutil.copyfile(os.path.join(path_smb,'msi2pi_h2o2.csv'),os.path.join(titration_path, 'msi2pi_h2o2.csv'))
            
            start_realsense(fname=out['save_filename'],
                            folder=titration_path,
                            frame_interval=1,#seconds
                            stop=0.001,#hours
                            take_image=True,
                            sensitivity = 150)
            
            cleanup(titration_path,out['save_filename'])

        # stop
        if out['file'] and out['mode'] == 'exit':
            break

except KeyboardInterrupt:
    print('interrupted')
    delete_files(path_smb)
    #if os.path.exists(os.path.join(path_smb,'msi2pi_h2o2.csv')) : os.remove(os.path.join(path_smb,'msi2pi_h2o2.csv'))
    
delete_files(path_smb)

interrupted


# Titration Analysis

In [3]:
# save raw data
cls2.analyze_titration(os.path.join(path_cam, 'titration'), vol_step = 5)

# visualize
plot_titration(os.path.join(path_cam, 'titration'), vol_step = 5)

NameError: name 'cls2' is not defined

In [2]:
path_smb = r'C:\Users\scrc112\Desktop\work\biplab\OpenTron\DriveSMB'
path_cam = r'C:\Users\scrc112\Desktop\work\yuan\protcol_h2o2_test'
prest_path = r'C:\Users\scrc112\Desktop\work\yuan\protcol_h2o2_test\preestimation'
mask_path = r'C:\Users\scrc112\Desktop\work\yuan\protcol_h2o2_test\mask'

with open(os.path.join(path_smb,'ot2_transfer_config_h2o2.json'),'r') as f:
    data = json.load(f)
    
tcdict = data['transfer_3']['locs']

cls2 = wellsegment_h2o2.load_pkl(prest_path,'COSTAR-pre23.png',os.path.join(mask_path, 'mask.pkl'))
cls2.analyze_prestimation(tcdict)

dict_keys(['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9', 'A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10', 'A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11', 'A12', 'B12', 'C12', 'D12', 'E12', 'F12', 'G12', 'H12'])
34.65637235990994
19.016136597142076
34.83578909671668


# Fitting Analysis

In [1]:
cls2

NameError: name 'cls2' is not defined